# Neural Network

Let's try to "learn" the "exclusive or" function. Unlike "and" or "or", it cannot be learned by a single neuron/perceptron, because it is not linearly seperable.

In [1]:
import numpy as np

For two inputs, xor is true when one of them is true, and false otherwise.

In [2]:
training_inputs = [
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0]
]

training_labels = [0, 1, 1, 0]

In [3]:
def model(inputs, labels, learning_rate, iterations):
    return None

In [4]:
parameters = model(training_inputs, training_labels, 0.25, 100)
print(parameters)

None
